## Dataset prep debugging

In [ ]:
import pandas as pd
import numpy as np
from gluonts.dataset.common import ListDataset, TrainDatasets, MetaData
from gluonts.dataset.field_names import FieldName

X = pd.read_csv('X_fd.csv', index_col=0)

%cd /mnt/c/Users/resha/Documents/Github/balancing_framework/notebooks_gluonts/

In [ ]:
%cd /mnt/c/Users/resha/Documents/Github/balancing_framework/
from monash_data_utils import convert_tsf_to_dataframe, monash_df_to_gluonts_train_datasets
import os
dataset = None
monash_dir = "monash_data"
dataset_name = 'm4_daily_dataset'

if os.path.exists(f"{monash_dir}/{dataset_name}.tsf"):
    loaded_data, frequency, forecast_horizon, contain_missing_values, contain_equal_length = convert_tsf_to_dataframe(f"{monash_dir}/{dataset_name}.tsf")
    if forecast_horizon is None: forecast_horizon = 24
    dataset = monash_df_to_gluonts_train_datasets(loaded_data, frequency, forecast_horizon)

14

## Debugging rest

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/mnt/c/Users/resha/Documents/Github/balancing_framework


In [ ]:
%load_ext autoreload
%autoreload 2
%cd /mnt/c/Users/resha/Documents/Github/balancing_framework/
from gluonts.dataset.repository import get_dataset, dataset_names
from gluonts.dataset.util import to_pandas

import pickle
import pandas as pd
import numpy as np
import json
import time
import argparse
from tqdm import tqdm

from framework import run_measurements, viz
from fracdiff import frac_diff_bestd
from monash_data_utils import convert_tsf_to_dataframe, monash_df_to_gluonts_train_datasets
import os

# parser = argparse.ArgumentParser()
# parser.add_argument('-f','--forms', type=str, nargs='+', help="Data forms, one or more of: ['fd','fod','o'] corresponding to ['frac_diff' , 'first_order_diff', 'original']")
# parser.add_argument('-d','--dataset', type=str, help="Dataset, matching one pullable from gluonts")
# args = parser.parse_args()

# data_forms = [ f.lower() for f in args.forms ]
# dataset_name = args.dataset
# print(f'Running measurements with params: format={data_forms}, dataset_name={dataset_name}')

data_forms = ['o']
dataset_name = 'm4_daily_dataset'
model_name = 'transformer'


valid_forms = ['fd' , 'fod', 'o', 'ta_o', 'ta_fod', 'ta_frac_diff', 'auto']
for data_form in data_forms:
    if data_form not in valid_forms:
        raise ValueError(f'The data_form arg must be one of: {valid_forms}')

if data_forms[0] == 'auto': form_sets = [['o', 'fd' , 'fod'], ['o', 'fd'], ['o', 'fod'], ['o']] 
else: form_sets = [data_forms]


start = time.time()


####################################### Load Data ##############################################################################################

# print(f"Available datasets: {dataset_names}")
dataset = None
monash_dir = "monash_data"

if os.path.exists(f"{monash_dir}/{dataset_name}.tsf"):
    loaded_data, frequency, forecast_horizon, contain_missing_values, contain_equal_length = convert_tsf_to_dataframe(f"{monash_dir}/{dataset_name}.tsf")
    if forecast_horizon is None: forecast_horizon = 24
    dataset = monash_df_to_gluonts_train_datasets(loaded_data, frequency, forecast_horizon)

if dataset is None:
    if dataset_name in dataset_names:
        dataset = get_dataset(dataset_name)
    else:
        raise ValueError(f"Dataset {dataset_name} not found in gluonts availables or local monash files.")
    

series_num = -1



for entry in tqdm(dataset.test):
    series_num += 1
    # for using a single pre-picked series
    if series_num != 2774:
        continue
    if series_num > 2774:
        break

    ####################################### Prep Feats & Labels  ####################################################################################

    def derive_feats(df):
        # Define window sizes for rolling calculations
        window_sizes = [2, 3, 4, 5]

        # Calculate rolling window features for different window sizes
        for window_size in window_sizes:
            df[f'rolling_mean_{window_size}'] = df['values'].rolling(window=window_size).mean()
            df[f'rolling_sum_{window_size}'] = df['values'].rolling(window=window_size).sum()
            df[f'rolling_min_{window_size}'] = df['values'].rolling(window=window_size).min()
            df[f'rolling_max_{window_size}'] = df['values'].rolling(window=window_size).max()
            df[f'rolling_std_{window_size}'] = df['values'].rolling(window=window_size).std()

        # Calculate additional features
        df['lag_1'] = df['values'].shift(1)  # Previous value
        df['lag_2'] = df['values'].shift(2)  # Value two steps back
        df['lag_3'] = df['values'].shift(3)  # Value three steps back
        df['lag_4'] = df['values'].shift(4)  # Value four steps back
        df['difference'] = df['values'].diff()  # Difference from the previous value
        df['percentage_change'] = df['values'].pct_change()  # Percentage change from the previous value
        df['cumulative_sum'] = df['values'].cumsum()  # Cumulative sum
        df['cumulative_mean'] = df['values'].expanding().mean()  # Cumulative mean
        df['cumulative_max'] = df['values'].cummax()  # Cumulative maximum
        df['cumulative_min'] = df['values'].cummin()  # Cumulative minimum
        df.dropna(inplace=True)

        return df.reset_index(drop=True)
    
    df = pd.DataFrame(entry['target'])
    df['label'] = 0
    df.rename(columns={0: 'values'}, inplace=True)

    df = derive_feats(df)
    df = df.replace([np.inf, -np.inf], np.nan).dropna()



    ####################################### Prep Data Forms ##############################################################################################


    for data_forms in form_sets:
        print(f'Data forms: {data_forms}')      
            
        X = pd.DataFrame()
        y = df['label']

        for data_form in data_forms:
            if data_form == 'fd':
                df_fd, fd_change_pct = frac_diff_bestd(df.drop(columns=['label']) )
                df_fd.dropna(inplace=True)
                df_fd.reset_index(drop=True, inplace=True)
                X = X.join(df_fd.add_suffix(f'_{data_form}'), how='outer')
                del df_fd

            elif data_form == 'o':
                X = X.join(df.drop(columns=['label']).add_suffix(f'_{data_form}'), how='outer')
            
            elif data_form == 'fod':
                # if wanted to omit cols from diff
                # diff = df.drop(['volume', 'transactions', 'label'], axis=1).diff()
                # diff = diff.join(df[['volume', 'transactions']])
                diff = df.drop(['label'], axis=1).diff().add_suffix(f'_{data_form}')
                X = X.join(diff, how='outer')
            # del df
            # elif data_form == 'ta_original':
            #     X = X.join(df_ta.drop(columns=['label']).add_suffix(f'_{data_form}'), how='outer')
            #     # del df_ta
            # elif data_form == 'ta_fod':
            #     diff = df_ta.drop(['label'], axis=1).diff().add_suffix(f'_{data_form}')
            #     X = X.join(diff, how='outer')
            #     del df_ta
            # elif data_form == 'ta_frac_diff':
            #     X = X.join(df_fd_ta.drop(columns=['label']).add_suffix(f'_{data_form}'), how='outer')
            #     del df_fd_ta
        
        X.dropna(inplace=True)
        # drop y with index not in X
        y = y[y.index.isin(X.index)]

        # X.to_csv(f'X_{data_form}.csv')
        
        # exit()

        ####################################### Run Framework ##############################################################################################

        # lower bounds to chunk size
        # min to run, chunk_size > X.shape(1) * 10
        # min for 10 splits, chunk_size > X.shape(0) /10 ;;; can adjust this min, unsure atm how many series fails condition
        # upper bound to chunk size
        # max num of splits, 100, 100 > (X.shape(0) / chunk_size) therefore default to 1% chunk size, using the lower bound
        
        chunk_size = int(X.shape[0] * 0.1)
        min_splits = 3
        min_chunk_size = X.shape[1] * 10
        if chunk_size < min_chunk_size:
            chunk_size = min_chunk_size
        if chunk_size > (X.shape[0] / min_splits):
            print(f'series length {X.shape[0]} with {X.shape[1]} feats does not meet criteria of >=10*n_features and a resulting count of splits > min_splits={min_splits}.')
            pass
        # cold_start_size = 10_000
        num_runs = 10

        print(f'Running measurements with params: format={data_form},chunk_size={chunk_size}, num_runs={num_runs}, \
            dataset_name={dataset_name}, model_name={model_name}')

        a,c,p = run_measurements(X, y, chunk_size, dataset_name, model_name, num_runs=num_runs, frac_diff=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/mnt/c/Users/resha/Documents/Github/balancing_framework


  0%|          | 0/4227 [00:00<?, ?it/s]

Data forms: ['o']
Running measurements with params: format=o,chunk_size=420, num_runs=1,             dataset_name=m4_daily_dataset, model_name=transformer
RUNNING ADAPTATION MEASURE


[I 2025-09-07 18:23:54,801] A new study created in memory with name: no-name-a927fe54-86ff-4ec8-b9d3-6452e3cad549


Tuning run 1 of 10 chunks


...


100%|██████████| 100/100 [00:08<00:00, 12.00it/s, epoch=1/68, avg_epoch_loss=12.3]

1it [00:00, 34.46it/s, epoch=1/68, validation_avg_epoch_loss=12.3]

100%|██████████| 100/100 [00:08<00:00, 12.13it/s, epoch=2/68, avg_epoch_loss=11.6]

1it [00:00, 39.01it/s, epoch=2/68, validation_avg_epoch_loss=11.6]

100%|██████████| 100/100 [00:08<00:00, 11.43it/s, epoch=3/68, avg_epoch_loss=10.9]

1it [00:00, 37.24it/s, epoch=3/68, validation_avg_epoch_loss=10.8]

100%|██████████| 100/100 [00:08<00:00, 11.77it/s, epoch=4/68, avg_epoch_loss=10.2]

1it [00:00, 36.17it/s, epoch=4/68, validation_avg_epoch_loss=10.1]

100%|██████████| 100/100 [00:08<00:00, 11.96it/s, epoch=5/68, avg_epoch_loss=9.62]

1it [00:00, 37.67it/s, epoch=5/68, validation_avg_epoch_loss=9.42]

100%|██████████| 100/100 [00:08<00:00, 11.94it/s, epoch=6/68, avg_epoch_loss=9.08]

1it [00:00, 41.20it/s, epoch=6/68, validation_avg_epoch_loss=8.83]

100%|██████████| 100/100 [00:08<00:00, 11.82it/s, epoch=7/68, avg_epoch_loss=8.67]

1it

Tuning run 10 of 10



100%|██████████| 100/100 [00:09<00:00, 10.92it/s, epoch=1/85, avg_epoch_loss=9.7]

1it [00:00, 33.41it/s, epoch=1/85, validation_avg_epoch_loss=8.85]

100%|██████████| 100/100 [00:09<00:00, 11.00it/s, epoch=2/85, avg_epoch_loss=8.78]

1it [00:00, 36.79it/s, epoch=2/85, validation_avg_epoch_loss=9.44]

100%|██████████| 100/100 [00:09<00:00, 10.71it/s, epoch=3/85, avg_epoch_loss=8.69]

1it [00:00, 34.22it/s, epoch=3/85, validation_avg_epoch_loss=9.45]

100%|██████████| 100/100 [00:09<00:00, 10.80it/s, epoch=4/85, avg_epoch_loss=8.66]

1it [00:00, 36.46it/s, epoch=4/85, validation_avg_epoch_loss=9.4]

100%|██████████| 100/100 [00:09<00:00, 10.59it/s, epoch=5/85, avg_epoch_loss=8.6]

1it [00:00, 35.32it/s, epoch=5/85, validation_avg_epoch_loss=9.29]

100%|██████████| 100/100 [00:09<00:00, 10.81it/s, epoch=6/85, avg_epoch_loss=8.59]

1it [00:00, 33.81it/s, epoch=6/85, validation_avg_epoch_loss=9.21]

100%|██████████| 100/100 [00:09<00:00, 10.88it/s, epoch=7/85, avg_epoch_loss=8.54]

1it [0

{'MSE': 230674.6607142857, 'abs_error': 7282.85009765625, 'abs_target_sum': 28499.556640625, 'abs_target_mean': 2035.6826171875, 'seasonal_error': 12.084676548617967, 'MASE': 43.04654545872059, 'MAPE': 0.25589360509599957, 'sMAPE': 0.22615540027618408, 'MSIS': 408.2793682804123, 'num_masked_target_values': 0.0, 'QuantileLoss[0.1]': 1473.31552734375, 'Coverage[0.1]': 0.0, 'QuantileLoss[0.5]': 7282.8504638671875, 'Coverage[0.5]': 1.0, 'QuantileLoss[0.9]': 4216.798999023436, 'Coverage[0.9]': 1.0, 'RMSE': 480.2860196948124, 'NRMSE': 0.23593364488142843, 'ND': 0.25554257525798957, 'wQuantileLoss[0.1]': 0.05169608586975689, 'wQuantileLoss[0.5]': 0.25554258810769603, 'wQuantileLoss[0.9]': 0.14796016135256487, 'mean_absolute_QuantileLoss': 4324.321663411458, 'mean_wQuantileLoss': 0.15173294511000593, 'MAE_Coverage': 0.5, 'OWA': nan}
Optimization Time: 13.10224846591785 minutes
Training run 10 of 10
Running with tuned params: {'model_dim_num_heads_pair': (90, 2), 'inner_ff_dim_scale': 5, 'embed


100%|██████████| 100/100 [00:09<00:00, 10.98it/s, epoch=1/85, avg_epoch_loss=8.88]

1it [00:00, 23.74it/s, epoch=1/85, validation_avg_epoch_loss=9.04]

100%|██████████| 100/100 [00:08<00:00, 11.23it/s, epoch=2/85, avg_epoch_loss=8.71]

1it [00:00, 36.46it/s, epoch=2/85, validation_avg_epoch_loss=8.95]

100%|██████████| 100/100 [00:08<00:00, 11.29it/s, epoch=3/85, avg_epoch_loss=8.65]

1it [00:00, 39.73it/s, epoch=3/85, validation_avg_epoch_loss=8.84]

100%|██████████| 100/100 [00:08<00:00, 11.20it/s, epoch=4/85, avg_epoch_loss=8.61]

1it [00:00, 36.48it/s, epoch=4/85, validation_avg_epoch_loss=8.68]

100%|██████████| 100/100 [00:08<00:00, 11.12it/s, epoch=5/85, avg_epoch_loss=8.54]

1it [00:00, 37.39it/s, epoch=5/85, validation_avg_epoch_loss=8.57]

100%|██████████| 100/100 [00:09<00:00, 10.83it/s, epoch=6/85, avg_epoch_loss=8.49]

1it [00:00, 32.12it/s, epoch=6/85, validation_avg_epoch_loss=8.45]

100%|██████████| 100/100 [00:09<00:00, 11.00it/s, epoch=7/85, avg_epoch_loss=8.43]

1it

In [ ]:
X

In [18]:
np.array(X2.T).shape

(30, 4207)

In [16]:
X2

,rolling_mean_2_o,rolling_sum_2_o,rolling_min_2_o,rolling_max_2_o,rolling_std_2_o,rolling_mean_3_o,rolling_sum_3_o,rolling_min_3_o,rolling_max_3_o,rolling_std_3_o,...,lag_1_o,lag_2_o,lag_3_o,lag_4_o,difference_o,percentage_change_o,cumulative_sum_o,cumulative_mean_o,cumulative_max_o,cumulative_min_o
0,590.245026,1180.490051,587.900024,592.590027,3.316333,594.873352,1784.620056,587.900024,604.130005,8.352440,...,592.590027,604.130005,620.919983,637.349976,-4.690002,-0.007914,3.042890e+03,608.578003,637.349976,587.900024
1,589.380005,1178.760010,587.900024,590.859985,2.093008,590.450012,1771.350037,587.900024,592.590027,2.371727,...,587.900024,592.590027,604.130005,620.919983,2.959961,0.005035,3.633750e+03,605.625000,637.349976,587.900024
2,587.664978,1175.329956,584.469971,590.859985,4.518423,587.743327,1763.229980,584.469971,590.859985,3.197888,...,590.859985,587.900024,592.590027,604.130005,-6.390015,-0.010815,4.218220e+03,602.602853,637.349976,584.469971
3,580.039978,1160.079956,575.609985,584.469971,6.264956,583.646647,1750.939941,575.609985,590.859985,7.658265,...,584.469971,590.859985,587.900024,592.590027,-8.859985,-0.015159,4.793830e+03,599.228745,637.349976,575.609985
4,582.484985,1164.969971,575.609985,589.359985,9.722718,583.146647,1749.439941,575.609985,589.359985,6.969865,...,575.609985,584.469971,590.859985,587.900024,13.750000,0.023888,5.383189e+03,598.132216,637.349976,575.609985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4202,2066.084961,4132.169922,2065.909912,2066.260010,0.247556,2065.853271,6197.559814,2065.389893,2066.260010,0.437815,...,2066.260010,2065.389893,2049.219971,2049.340088,-0.350098,-0.000169,4.212416e+06,1001.287361,2091.830078,423.220001
4203,2071.164917,4142.329834,2065.909912,2076.419922,7.431699,2069.529948,6208.589844,2065.909912,2076.419922,5.969460,...,2065.909912,2066.260010,2065.389893,2049.219971,10.510010,0.005087,4.214493e+06,1001.542859,2091.830078,423.220001
4204,2080.395020,4160.790039,2076.419922,2084.370117,5.621637,2075.566650,6226.699951,2065.909912,2084.370117,9.259635,...,2076.419922,2065.909912,2066.260010,2065.389893,7.950195,0.003829,4.216578e+06,1001.800123,2091.830078,423.220001
4205,2070.250000,4140.500000,2056.129883,2084.370117,19.968861,2072.306641,6216.919922,2056.129883,2084.370117,14.562521,...,2084.370117,2076.419922,2065.909912,2066.260010,-28.240234,-0.013549,4.218634e+06,1002.050558,2091.830078,423.220001
